In [179]:
post = {'id': 0, 'coords': (0, 2), 'descr': 'Почтовое отделение'}
point1 = {'id': 1, 'coords': (2, 5), 'descr': 'Ул. Грибоедова, 104/25'}
point2 = {'id': 2, 'coords': (5, 2), 'descr': 'Ул. Бейкер стрит, 221б'}
point3 = {'id': 3, 'coords': (6, 6), 'descr': 'Ул. Большая Садовая, 302-бис'}
point4 = {'id': 4, 'coords': (8, 3), 'descr': 'Вечнозелёная Аллея, 742'}

point_list = [post, point1, point2, point3, point4]
point_set = set([point_['id'] for point_ in point_list[1:]])
all_routes = []
ALLOWED_DELTA = 0.001

In [180]:
def create_distance_array(point_list_):
    calc_dist = lambda a, b : ((a['coords'][0] - b['coords'][0]) ** 2 + (a['coords'][1] - b['coords'][1]) ** 2) ** 0.5
    len_ = len(point_list_)
    dist_array_ = [[0 for col_ in range(len_)] for raw_ in range(len_)]
    for start_point in point_list_:
        s_ = start_point['id']
        for finish_point in point_list_[s_+1:]:
            f_ = finish_point['id']
            dist_array_[s_][f_] = dist_array_[f_][s_] = calc_dist(point_list_[s_], point_list_[f_])
    return dist_array_

def multiply_list(set_: set={}, list_: list=[]):
    output_list_ = []
    for item_ in (s_ for s_ in set_):
        output_list_.append(list_ + [item_])
    return output_list_

def find_routes(set_: set={}, list_: list=[]):
    result_list_ = []
    if len(list_) > 0:
        if len(list_[0]) < len(set_):
            tmp_list_ = []
            for id_list_ in list_:
                tmp_set_ = set_.copy()
                for id_ in id_list_:
                    tmp_set_.remove(id_)
                tmp_list_ += multiply_list(tmp_set_, id_list_)
            result_list_ = tmp_list_
            find_routes(set_, result_list_)
        else:
            global all_routes
            all_routes = list_
    else:
        result_list_ += multiply_list(set_)
        find_routes(set_, result_list_)

In [181]:
def calc_distance(route_list_, distance_array_):
    distance = (distance_array_[0][route_list_[0]] + distance_array_[0][route_list_[-1]])
    for d_ in range(len(route_list_) - 1):
        distance += distance_array_[route_list_[d_]][route_list_[d_+1]]
    return distance

def get_shorty(route_list_, point_list_, distance_array_):
    min_route_ = []
    min_dist_ = 0.0
    for route_ in route_list_:
        length_ = calc_distance(route_, distance_array_)
        #####print(length_)
        if min_dist_:
            if length_ < min_dist_:
                min_dist_ = length_
                min_route_ = [route_]
                #####print('minimum check')
        else:
            min_dist_ = length_
    for route_ in route_list_:
        if route_ != min_route_[0]:
            length_ = calc_distance(route_, distance_array_)
            if (length_ - min_dist_) ** 2 < ALLOWED_DELTA:
                min_route_.append(route_)
    return min_route_

In [182]:
def write_string(list_, point_list_, distance_array_):
    dist_sum_ = 0.0
    list_ = [0] + list_ + [0]
    output_str_ = str(point_list_[0]['coords'])
    for r_ in range(len(list_) - 2):
        dist_sum_ += distance_array_[list_[r_]][list_[r_+1]]
        #####print('DIST', distance_array_[list_[r_]][list_[r_+1]])
        #####print('DIST_SUM', dist_sum_)
        p_ = str(point_list_[list_[r_+1]]['coords'])
        d_ = str(dist_sum_)
        output_str_ += ' -> {}[{}]'.format(p_, d_)
    dist_sum_ += distance_array_[list_[-2]][0]
    #####print('DIST', distance_array_[list_[r_]][list_[r_+1]])
    #####print('DIST_SUM', dist_sum_)
    p_ = str(point_list_[0]['coords'])
    d_ = str(dist_sum_)
    output_str_ += ' -> {}[{}] = {}'.format(p_, d_, d_)
    return output_str_

def output_result(short_list_, point_list_, distance_array_):
    output_str_ = u'Кратчайший маршрут:\n'
    output_str_ += write_string(short_list_[0], point_list_, distance_array_)
    if len(short_list_) > 1:
        output_str_ += u'\nАльтернативные кратчайшие маршруты:\n'
        for alt_list_ in short_list_[1:]:
            output_str_ += write_string(alt_list_, point_list_, distance_array_)
    return output_str_

In [186]:
distance_array = create_distance_array(point_list)
#####for k_ in distance_array:
    #####print(k_)
find_routes(point_set)
minimals = get_shorty(all_routes, point_list, distance_array)
#####print('ITOGO =', minimals)
print(output_result(minimals, point_list, distance_array))

Кратчайший маршрут:
(0, 2) -> (2, 5)[3.605551275463989] -> (6, 6)[7.728656901081649] -> (8, 3)[11.334208176545639] -> (5, 2)[14.496485836714019] -> (0, 2)[19.49648583671402] = 19.49648583671402
Альтернативные кратчайшие маршруты:
(0, 2) -> (5, 2)[5.0] -> (8, 3)[8.16227766016838] -> (6, 6)[11.76782893563237] -> (2, 5)[15.890934561250031] -> (0, 2)[19.49648583671402] = 19.49648583671402
